# Run script - Google Earth Engine 

## This script is adpated from F. van Oorschot: https://github.com/fvanoorschot/global_sr_module

In this script we will gather catchment data from satellite products (e.g. tree cover, NDVI, elevation) using Google Earth Engine (GEE). GEE allows us to directly use satellite data, avoiding the struggle of downloading them. Before using it, you need to create an account: https://signup.earthengine.google.com/#!/

This scripts only works in the conda environment **Snow_module_env**. In this environment all required packages are available. If you have **not** installed and activated this environment before opening this script, you should check the installation section in the *README* file. 


### 1. import all the required packages.

In [1]:
# import packages
import ee
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import os
import glob
from pathlib import Path
from pathos.threading import ThreadPool as Pool
import datetime
from dateutil.relativedelta import relativedelta

Before using the Earth Engine API or earthengine command line tool, you must perform a one-time authentication that authorizes access to Earth Engine on behalf of your Google account. Below you run the authentication command. A URL will be provided that generates an authorization code upon agreement. Copy the authorization code and enter it in the box below.

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

After authentication we can import all the python functions defined in the scripts *f_earth_engine_edits.py*.

In [4]:
from f_earth_engine_edits import *

### 2. Define working and data directories
Here we define the working directory, where all the scripts and output are saved.

We also define the data directory where you have the following subdirectories:

/data/forcing/*netcdf forcing files*\
/data/shapes/*catchment shapefiles*\
/data/gsim_discharge/*EStreams discharge timeseries*

In [5]:
# Check current working directory (helpful when filling in work_dir below)
os.getcwd()

'C:\\Users\\juliarudlang\\OneDrive - Delft University of Technology\\Scripts\\Snow_module'

In [6]:
# define your script working directory
work_dir=Path("C:\\Users\\juliarudlang\\OneDrive - Delft University of Technology\\Scripts\\Snow_module")
# define your data directory (optional)
#data_dir="U:\Hydrological Data\EStreams\Subset_v2_290324\Subset_240424"

### 3. Load your list of catchment IDs
Here we load the list of catchments IDs from the catchmen boundarie file

In [7]:
catchment_boundaries = gpd.read_file(r'U:\Hydrological Data\EStreams\Subset_v2_290324\Subset_240424\Catchment_Boundaries\estreams_catchments_subset.shp')
#catchment_boundaries

In [8]:
catchment_boundaries_to_take = ['basin_id', 'geometry'] 
catchment_B = catchment_boundaries[catchment_boundaries_to_take]

In [9]:
catch_id_list = catchment_B["basin_id"].tolist()

In [10]:
len(catch_id_list)

7264

### 4. Earth Engine treecover
We are interested in the treecover in a catchment. For this we use the MODIS treecover data (https://modis.gsfc.nasa.gov/data/dataprod/mod44.php). This product includes the percentage tree cover, non tree cover, and bare soil on a 250x250 m grid. Here we regrid the tree cover to a 1x1 km grid (to reduce computational costs), average the values over the time period of interest and extract the catchment statistics (mean, max, min and std).

First we create the output directory:

In [26]:
# make output directory
if not os.path.exists(f'U:/Hydrological Data/EStreams/Subset_v2_290324/Subset_240424/Snow_Module/output/earth_engine_timeseries/treecover'):
    os.makedirs(f'U:/Hydrological Data/EStreams/Subset_v2_290324/Subset_240424/Snow_Module/output/earth_engine_timeseries/treecover')

Now we run the *preprocess_treecover_data* and *catchment_treecover* functions from the *f_earth_engine_edits.py* script. The output is a dataframe with the treecover statistics for each catchment.

In [32]:
# Check if the geometry is valid
shapefile['is_valid'] = shapefile.is_valid

# Print invalid geometries
if not shapefile['is_valid'].all():
    print(shapefile[~shapefile['is_valid']])

In [70]:
# define your time period
start_date = '1980-01-01'
end_date = '2023-12-31'

# define your directories
#shape_dir = Path(f'{data_dir}/shapes/')
shape_dir = Path(f'U:/Hydrological Data/EStreams/Subset_v2_290324/Subset_240424/CB_Single_v4')
out_dir = Path(f'U:/Hydrological Data/EStreams/Subset_v2_290324/Subset_240424/Snow_Module/output/earth_engine_timeseries/treecover')

# preprocess your modis satellite data for your time period (interpolation and averaging)
(MOD44B_tree_res, MOD44B_nontree_res) = preprocess_treecover_data(start_date,end_date)

# loop over catch ids
for catch_id in catch_id_list:
    # extract catchment values and store in dataframe
    catchment_treecover(MOD44B_tree_res, MOD44B_nontree_res, catch_id, shape_dir, out_dir)

In [66]:
# print treecover statistics for catchment [0] in catch_id_list
catch_id = catch_id_subset[0]
c = pd.read_csv(f'{out_dir}/{catch_id}.csv',index_col=0)
c.head()

,max_tc,mean_tc,min_tc,std_tc,max_ntc,mean_ntc,min_ntc,std_ntc,mean_nonveg
SKGR0017,40.217534,34.725346,30.952223,2.268262,52.822187,49.537363,44.635987,2.043585,15.737291


### 5. Earth Engine DEM

In [11]:
shape_dir =  Path(f'U:/Hydrological Data/EStreams/Subset_v2_290324/Subset_240424/CB_Single_v4')

In [11]:
shape_dir # here we have separated the the catchmen boundatires to single files for each boundary

WindowsPath('U:/Hydrological Data/EStreams/Subset_v2_290324/Subset_240424/CB_Single_v4')

In [74]:
work_dir_list = [work_dir]*len(catch_id_list)

In [13]:
catch_id_list[0] # check

'AT000001'

In [13]:
work_dir_list = [shape_dir]*len(catch_id_list[0])
work_dir_list

[WindowsPath('U:/Hydrological Data/EStreams/Subset_v2_290324/Subset_240424/CB_Single_v4'),
 WindowsPath('U:/Hydrological Data/EStreams/Subset_v2_290324/Subset_240424/CB_Single_v4'),
 WindowsPath('U:/Hydrological Data/EStreams/Subset_v2_290324/Subset_240424/CB_Single_v4'),
 WindowsPath('U:/Hydrological Data/EStreams/Subset_v2_290324/Subset_240424/CB_Single_v4'),
 WindowsPath('U:/Hydrological Data/EStreams/Subset_v2_290324/Subset_240424/CB_Single_v4'),
 WindowsPath('U:/Hydrological Data/EStreams/Subset_v2_290324/Subset_240424/CB_Single_v4'),
 WindowsPath('U:/Hydrological Data/EStreams/Subset_v2_290324/Subset_240424/CB_Single_v4'),
 WindowsPath('U:/Hydrological Data/EStreams/Subset_v2_290324/Subset_240424/CB_Single_v4')]

In [15]:
shape_dir

WindowsPath('U:/Hydrological Data/EStreams/Subset_v2_290324/Subset_240424/CB_Single_v4')

In [12]:
elevation_zones(catch_id_list[0],shape_dir)

In [13]:
elevation_stats(catch_id_list[0],shape_dir)

In [79]:
len(catch_id_list)

7264

In [83]:
# Check to see if it find all the shapefiles
for catch_id in catch_id_list:
    files = glob.glob(f'{shape_dir}/{catch_id}.shp')
    if not files:
        raise FileNotFoundError(f"Shapefile for catch_id {catch_id} not found in {shape_dir}.")
    f = files[0]

In [14]:
# run elevation stats
#catch_list = np.genfromtxt(f'{work_dir}/output/gsim_aus_catch_id_list_lo_sel.txt',dtype='str')[:]
work_dir_list = [shape_dir]*len(catch_id_list)
run_function_parallel_elevation_stats(catch_id_list,work_dir_list)

In [7]:
# check which catchments are missing
catch_list = np.genfromtxt(f'{work_dir}/output/gsim_aus_catch_id_list_lo_sel.txt',dtype='str')[:]
el_id_list=[]
for filepath in glob.iglob(f'{work_dir}/output/elevation/stats/*.csv'):
    f = os.path.split(filepath)[1] # remove full path
    f = f[:-4] # remove .year extension
    el_id_list.append(f)
dif = list(set(catch_list) - set(el_id_list))
len(dif)

523

In [9]:
catch_list = dif[:]
work_dir_list = [work_dir]*len(catch_list)
run_function_parallel_elevation_stats(catch_list,work_dir_list)

In [16]:
catch_id_list[-14:]

['SKGR0008',
 'SKGR0009',
 'SKGR0010',
 'SKGR0011',
 'SKGR0012',
 'SKGR0013',
 'SKGR0014',
 'SKGR0015',
 'SKGR0016',
 'SKGR0017',
 'SKGR0018',
 'SKGR0021',
 'UAGR0001',
 'UAGR0002']

In [15]:
# run elevation zones
# catch_list = np.genfromtxt(f'{work_dir}/output/snow/catch_id_list_snow_t_and_p.txt',dtype='str')[:]
#catch_list = np.genfromtxt(f'{work_dir}/output/us_camels_ids.txt',dtype='str')[:]
work_dir_list = [shape_dir]*len(catch_id_list)
run_function_parallel_elevation_zones(catch_id_list,work_dir_list)

EEException: Computation timed out.

In [20]:
# check which catchments are missing
#catch_list = np.genfromtxt(f'{work_dir}/output/gsim_aus_catch_id_list_lo_sel.txt',dtype='str')[:]
el_id_list=[]
for filepath in glob.iglob(f'U:/Hydrological Data/EStreams/Subset_v2_290324/Subset_240424/Snow_Module/output/elevation/el_zones/*.csv'):
    f = os.path.split(filepath)[1] # remove full path
    f = f[:-4] # remove .year extension
    el_id_list.append(f)
dif = list(set(catch_id_list) - set(el_id_list))
len(dif)

14

In [21]:
dif

['GB000565',
 'GB000570',
 'GB000560',
 'GB000568',
 'GB000569',
 'GB000564',
 'GB000567',
 'GB000574',
 'GB000571',
 'GB000563',
 'GB000562',
 'GB000561',
 'GB000572',
 'GB000566']

In [22]:
work_dir_list = [shape_dir]*len(dif)
run_function_parallel_elevation_zones(dif,work_dir_list)

In [25]:
#combine all files with elevation stats 
files = glob.glob(f"U:/Hydrological Data/EStreams/Subset_v2_290324/Subset_240424/Snow_Module/output/elevation/stats/ele*")
li=[] #empty list
for filename in files:
    df = pd.read_csv(filename, index_col=0) #read file as dataframe
    li.append(df) #append file to list
f = pd.concat(li, axis=0) #concatenate lists
f.to_csv(f'U:/Hydrological Data/EStreams/Subset_v2_290324/Subset_240424/Snow_Module/output/elevation/all_catchment_elevation_stats.csv')

In [26]:
f

,max_ele,mean_ele,min_ele,std_ele
AT000001,3458.011475,1874.053101,424.948761,612.587965
DEBW0225,975.278625,454.239410,151.408020,195.671198
DEBW0392,424.737061,341.592743,221.880844,39.464092
AT000534,3065.465088,2168.972412,1215.075684,413.014223
DEBW0202,1197.211914,837.901123,338.554626,193.161427
...,...,...,...,...
SE000077,680.174622,350.427856,151.733353,95.450777
SE000287,302.150574,178.937134,62.254398,60.563829
SE000102,475.581940,89.411942,-1.232558,87.733073
SE000103,1225.378540,627.998169,397.175598,147.273815


In [28]:
#combine all files with elevation stats 
files = glob.glob(f"U:/Hydrological Data/EStreams/Subset_v2_290324/Subset_240424/Snow_Module/output/elevation/stats/slope*")
li=[] #empty list
for filename in files:
    df = pd.read_csv(filename, index_col=0) #read file as dataframe
    li.append(df) #append file to list
f = pd.concat(li, axis=0) #concatenate lists
f.to_csv(f'U:/Hydrological Data/EStreams/Subset_v2_290324/Subset_240424/Snow_Module/output/elevation/all_catchment_slope_stats.csv')

In [29]:
f

,max_slope,mean_slope,min_slope,std_slope
AT000001,50.049515,20.272083,0.065833,8.034362
DEBW0225,18.642906,8.964785,1.825206,3.451954
AT000534,40.751682,22.333809,3.755218,5.768330
DEBW0392,10.376493,3.625622,0.471354,1.680856
DEBW0202,27.611446,11.510314,0.380818,6.294215
...,...,...,...,...
SE000077,19.945879,2.702684,0.003885,2.009491
SE000287,4.869286,0.862721,0.001861,0.530233
SE000102,14.683974,1.283605,0.000166,1.236345
SE000103,29.101101,4.122903,0.018348,3.221416
